# Diet Lab
Note: this demo is based on the Fall 2019 ENGRI 1101 Lab 8 which is based off both chapter 2 of the AMPL book and section 6.2 of _Decision Making, Models and Algorithms: A First Course_ by Saul I. Gass.

Objectives:
- Introduce an historically important linear program
- Think about solving linear programs and tweaking solutions
- Introduce AMPL, a powerful computational tool for solving linear programs

In [1]:
# imports; make sure to import them!
import pandas as pd
from IPython.display import Image
from ortools.linear_solver import pywraplp as OR

## Part 1: The Diet Problem

Suppose that you have a choice of two cereals for breakfast: Krunchies (K) or Crispies (C). Breakfast is the
most important meal of the day, so you want to make sure you get sufficient Thiamine, Niacin, and caloric
intake for breakfast. Being a college student, you want to do so as cheaply as possible – taste gets thrown
into the wind! Price and nutritional info for these two cereals is summarized in the table below:

| Nutrients<br> | Amount of each nutrient (in mg)<br>in 1 ounce of Krunchies | Amount of each nutrient (in mg)<br>in 1 ounce of Crispies | Requirement of each<br>nutrient (in mg)
|-----------|---------------------------------|---------------------------------|----------------------|
| Thiamine  |                  0.1            |            0.25                 |       1.0            |
| Niacin    |                  1.0            |             0.25                |        5.0           |
| Calories  |                  110.0          |           120.0                 |          400.0       |
| Cost  |                 3.80                |           4.20                  |                      |


**Q:** Suppose you just ate Krunchies. How many ounces of Krunchies would you need to eat to satisfy the
three nutritional requirements? How much would this cost?

**A:**

**Q:** Suppose you just ate Crispies. How many ounces of Crispies would you need to eat to satisfy the three
nutritional requirements? How much would this cost?

**A:**

**Q:** Now let’s write an optimization problem to model this problem, just like you did all the way back on
the first lab/homework! Let K be a decision variable for the amount of Krunchies you eat and C be
a decision variable for the amount of Crispies you eat. Write an objective function, encoding that you
minimize total cost (as a function of K and C).

**A:**

**Q:** Write three constraints: one enforcing that you get at least 1 mg of Thiamine, one enforcing that you
get at least 5 mg of Niacin, and one enforcing that you get at least 400 calories. Also write constraints
that K and C are nonnegative.

**A:**

You just wrote a linear program! In the rest of this lab, we’ll think about how to solve it using Pyomo in Python

## Part 2: Solving The Cereal Diet Problem

On the previous pages, you were asked to write down a mathematical formulation. In class today we saw
that your model is actually “linear program:” it’s constraints are linear inequalities on decision variables (K
and C), and its objective function is to minimize a linear combination of decision variables. Hopefully you
wrote down:

$$
\begin{align}
\min &\quad 3.8K + 4.2C \\
\text{subject to} &\quad 0.1K + 0.25C &\geq 1 \\
&\quad 1K + 0.25C &\geq 5 \\
&\quad 110K + 120C &\geq 400 \\
&\quad 0 \leq K, C
\end{align}
$$


It’s important that you have inequalities saying that you get “at least” the required nutritional value.

**Q:** The feasible region is plotted below and has three marked points. Find the coordinates of points A, B,
and C. What is the corresponding diet at those points? What does it cost? (Hint: you already looked
at A and C in the previous page; to find point B, you might want to note that it’s where the lines
$0.1K + 0.25C = 1 K + 0.25C = 5$ meet.)

**A:**

**Q:** Based on your above work, what is the optimal solution to the linear program?

**A:**

**Q:** Now let’s try and solve this toy problem in AMPL. To open AMPL:

**A:**

We will now solve this LP in Python using OR-Tools

In [2]:
# define the model
diet = OR.Solver('diet', OR.Solver.GLOP_LINEAR_PROGRAMMING);

# decision variables
K = diet.NumVar(0, diet.infinity(), 'K');
C = diet.NumVar(0, diet.infinity(), 'C');

# objective function
diet.Minimize(3.8*K + 4.2*C);

# constraints
diet.Add(0.1*K + 0.25*C >= 1);
diet.Add(1*K + 0.25*C >= 5);
diet.Add(110*K + 120*C >= 400);

# solve and print solution
diet.Solve()
print('Objective =', diet.Objective().Value())
print('Solution:')
for v in diet.variables():
    print(v.name(),':', v.solution_value())

Objective = 26.222222222222225
Solution:
K : 4.444444444444445
C : 2.2222222222222228


**Q:** You just defined an LP model using Pyomo, solved it using a solver called gurobi, and then displayed the solutions for the variables. What did you find for $K, C$ and the total cost? Is this the same as
above?

**A:**

**Ans:**

## 3 A More Robust Diet

Let’s now expand our diet. Instead of looking at just breakfast, we’ll allow ourselves to eat more food. The
food options, together with the decision variable we’ll use, are:
- Beef ($X_{beef}$ )
- Chicken ($X_{chk}$)
- Fish ($X_{fish}$)
- Ham ($X_{ham}$)
- Mac & cheese ($X_{mch}$)
- Meat loaf ($X_{mtl}$)
- Spaghetti ($X_{spg}$)
- Turkey ($X_{tur}$)

As before, we’ll want to minimize cost. This time, we’ll want to meet nutritional requirements for vitamins
$A, C, B1$, and $B2$.

Costs (in dollars) and nutritional information are shown below. All costs/nutritional requirements are
per package. The values for nutritional requirements are percents. E.g. one package of beef provides 60% of
the daily requirement for vitamin A. In a day, you need 100% of your recommended amount of Vitamin A.

In [3]:
pd.read_csv('small_diet.csv')

,Name,Cost,Vitamin A,Vitamin C,Vitamin B1,Vitamin B2
0,Beef,3.19,60,20,10,15
1,Chk,2.59,8,0,20,20
2,Fish,2.59,8,10,15,10
3,Ham,2.89,40,40,35,10
4,Mch,1.89,15,35,15,15
5,Mtl,1.99,70,30,15,15
6,Spg,1.99,25,50,25,15
7,Tur,2.49,60,20,15,10


Let’s consider the problem of minimizing cost of a diet over an entire week.

**Q:** Write out the objective function for this problem (which is to minimize cost).

**A:**

**Q:** Write out the constraint indicating that you get enough Vitamin A in an entire week. Be careful about
the right hand side: what percent do you need over an entire week?

**A:**

**Q:** What solution does it give? How much does it cost, which food(s) do we eat, and how much of those food(s) do we eat?

**A:**

**Q:** Does the diet seem reasonable?

**A:**

Let’s add a constraint capping the maximum amount of mac and cheese we eat at 20 servings. (some explanation of how to do this ...)

**Q:** What is the new solution? How much more expensive is it? Does it seem healthier?

**A:**

**Q:** Suppose you wanted to add in 3 more food options (soylent, snickers, and coffee) and 10 more con-
straints (on vitamins B3, B5, B6, B9, B12, D, E, and K; on Sodium; and on the most important vitamin of all, caffeine). Or suppose that I asked you to instead do this problem with bread, meat,
potatoes, cabbage, milk, and gelatin. Write 2-3 sentences actually explaining what you’d have to do
in each of these two settings. Also briefly describe how tedious of a process it would be to edit this
model. Colorful (but not vulgar) language, metaphors, and famous quotes are all appropriate.

**A:**

## Model And Data

In [4]:
def Diet(foods, nutrients, integer=False):
    """Classic Diet Problem Model.
    
    Args:
        foods (pd.DataFrame): Foods with cost per serving and nutrients per serving.
        nutrients (pd.DataFrame): Nutrients with min and max bounds.
    """
    FOODS = list(foods.index)                                 # foods
    NUTRIENTS = list(nutrients.index)                         # nutrients
    c = foods['Cost'].to_dict()                               # cost per serving of food 
    f_min = foods['Min'].to_dict()                            # lower bound of food serving
    f_max = foods['Max'].to_dict()                            # upper bound of food serving
    n_min = nutrients['Min'].to_dict()                        # lower bound of nutrient
    n_max = nutrients['Max'].to_dict()                        # upper bound of nutrient  
    a = foods[list(nutrients.index)].transpose().to_dict()    # amt of nutrients per serving of food
    
    # define model
    m = OR.Solver('diet', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i in FOODS:
        if integer:
            x[i] = m.IntVar(f_min[i], f_max[i], 'x_%s' % (i)) 
        else:
            x[i] = m.NumVar(f_min[i], f_max[i], 'x_%s' % (i)) 
        
    # define objective function here
    m.Minimize(sum(c[i]*x[i] for i in FOODS))
    
    # enforce lower and upper bound on nutrients 
    for j in NUTRIENTS:
        m.Add(sum(a[i][j]*x[i] for i in FOODS) >= n_min[j])
        m.Add(sum(a[i][j]*x[i] for i in FOODS) <= n_max[j])
        
    return (m, x)  # return the model and the decision variables

In [5]:
# You do not need to do anything with this cell but make sure you run it!
def solve(m):
    """Used to solve a model m."""
    m.Solve()
    
    print('Objective =', m.Objective().Value())    
    return {var.name() : var.solution_value() for var in m.variables()}

In [6]:
foods = pd.read_csv('neos_foods.csv', index_col=0)
nutrients = pd.read_csv('neos_nutrients.csv', index_col=0)

In [7]:
m,x = Diet(foods, nutrients)
solve(m)

Objective = 0.9560078273216122


{'x_Frozen Broccoli': 0.0,
 'x_Carrots, Raw': 0.23581781088978732,
 'x_Celery, Raw': 0.0,
 'x_Frozen Corn': 0.0,
 'x_Lettuce, Iceberg,Raw': 0.0,
 'x_Peppers, Sweet, Raw': 0.0,
 'x_Potatoes, Baked': 3.5449447765210538,
 'x_Tofu': 0.0,
 'x_Roasted Chicken': 0.0,
 'x_Spaghetti W/ Sauce': 0.0,
 'x_Tomato,Red,Ripe,Raw': 0.0,
 'x_Apple, Raw, w/Skin': 0.0,
 'x_Banana': 0.0,
 'x_Grapes': 0.0,
 'x_Kiwifruit, Raw, Fresh': 0.0,
 'x_Oranges': 0.0,
 'x_Bagels': 0.0,
 'x_Wheat Bread': 0.0,
 'x_White Bread': 0.0,
 'x_Oatmeal Cookies': 0.0,
 'x_Apple Pie': 0.0,
 'x_Chocolate Chip Cookies': 0.0,
 'x_Butter, Regular': 0.0,
 'x_Cheddar Cheese': 0.0,
 'x_3.3% Fat, Whole Milk': 0.0,
 'x_2% Lowfat Milk': 0.0,
 'x_Skim Milk': 2.167849362594859,
 'x_Poached Eggs': 0.0,
 'x_Scrambled Eggs': 0.0,
 'x_Bologna, Turkey': 0.0,
 'x_Frankfurter, Beef': 0.0,
 'x_Ham, Sliced, Extralean': 0.0,
 'x_Kielbasa, Pork': 0.0,
 "x_Cap'N Crunch": 0.0,
 'x_Cheerios': 0.0,
 "x_Corn Flakes, Kellogg'S": 0.0,
 "x_Raisin Bran, Kellogg

In [8]:
foods = pd.read_csv('small_diet.csv').set_index('Name')
foods['Min'] = 0
foods['Max'] = float('inf')
nutrients = pd.DataFrame(index=['Vitamin A', 'Vitamin C',
                                'Vitamin B1', 'Vitamin B2'])
nutrients['Min']= 700 # 100% over an entire week (7 days)
nutrients['Max']= float('inf')

In [9]:
m,x = Diet(foods, nutrients)
solve(m)

Objective = 88.2


{'x_Beef': 0.0,
 'x_Chk': 0.0,
 'x_Fish': 0.0,
 'x_Ham': 0.0,
 'x_Mch': 46.66666666666667,
 'x_Mtl': 0.0,
 'x_Spg': 0.0,
 'x_Tur': 0.0}

In [10]:
m,x = Diet(foods, nutrients)
m.Add(x['Mch'] <= 20)
solve(m)

Objective = 89.778125


{'x_Beef': 0.0,
 'x_Chk': 17.187500000000004,
 'x_Fish': 0.0,
 'x_Ham': 0.0,
 'x_Mch': 19.999999999999996,
 'x_Mtl': 3.7500000000000004,
 'x_Spg': 0.0,
 'x_Tur': 0.0}